This notebook looks at different rotation parameters definitions, for each parametrization it outputs:
- A: transformation matrix from body to global
- G: Angular velocity matrix such that omega_global = G theta_dot (theta_dot is the derivative of the parameters)
- Gb: Angular velocity matrix such that omega_local = Gb theta_dot
- Ginv: Inverse of "G" matrix, such that  theta_dot = Ginv omega_global
- Gbinv: Inverse of "Gb" matrix, such that  theta_dot = Gbinv omega_local


This file was used to generate part of welib.yams.rotations.py

In [ ]:
# Ipython 
import matplotlib.pyplot as plt
from sympy.physics.vector import init_vprinting
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
init_vprinting(use_latex='mathjax', pretty_print=False)
# Sympy
import sympy as sp
from sympy import symbols, simplify, Function, expand_trig, cos, sin, diff, sqrt, Matrix, pi
from sympy.physics.mechanics import dynamicsymbols, ReferenceFrame, Point, inertia, RigidBody, KanesMethod
from sympy.physics.vector import get_motion_params, vlatex
# --- My packages
from welib.yams.yams_sympy_tools import *
# Main symbols
time = symbols('t')
phi_x, phi_y, phi_z = dynamicsymbols('phi_x,phi_y,phi_z')
dphi_x, dphi_y, dphi_z = diff(phi_x,time), diff(phi_y,time), diff(phi_z, time)
phi, theta, psi = dynamicsymbols('phi,theta,psi')
dphi, dtheta, dpsi = diff(phi,time), diff(theta,time), diff(psi, time)
omega_x, omega_y, omega_z = dynamicsymbols('omega_x, omega_y, omega_z')

In [ ]:
# Defining coordinate systems / frames
e_E = ReferenceFrame('E') # Earth/Ground
#XYZg = e_E.orientnew('T', 'Space', (phi_x, phi_y, phi_z), 'XYZ')
#e_T = e_E.orientnew('T', 'DCM', DCM)
#e_T0.dcm(e_E) # From Inertial to Tower
#omega_TE = e_T.ang_vel_in(e_E)  # Angular velocity of tower wrt to Earth (omega_T-omega_F)
#omega_TE.to_matrix(e_E).simplify()


def framemat(frame, e_E, thetas_sorted):
    theta_dot = [diff(var,time) for var in thetas_sorted]
    theta_ddot = [diff(var,time) for var in theta_dot]
    A = frame.dcm(e_E).T # From Frame to Global
    omega = frame.ang_vel_in(e_E)  # Angular velocity of frame wrt to Earth
    omega_g=omega.express(e_E).to_matrix(e_E).simplify()
    omega_t=omega.express(frame).to_matrix(frame).simplify()
    G     = omega_g.jacobian(theta_dot).simplify()
    Gbar  = omega_t.jacobian(theta_dot).simplify()
    Ginv  = G.inv().simplify()
    Gbinv = Gbar.inv().simplify()
    
    #alpha = diff(omega, time, e_E) 
    #alpha_g=alpha.express(e_E).to_matrix(e_E).simplify()
    #dG     = alpha_g.jacobian(theta_ddot).simplify() # This should equal G
    #if sum(G- dG)!=0:
    #    raise Exception('G is not the same for omega and alpha for {}')
    
    return A, omega, omega_g, omega_t, G, Gbar, Ginv, Gbinv


print('----------')
frame = e_E.orientnew('T', 'Body', (phi_x, phi_y, phi_z) , 'XYZ'); 
A, omega, omega_g, omega_t, G, Gbar, Ginv, Gbinv = framemat(frame, e_E, (phi_x, phi_y, phi_z))
omega_g
G

In [ ]:
def cleanPySmallMat(expr, varname='R', indent=0, replDict=None, noTimeDep=True): 
    def cleanPyAtom(atom):
        s=repr(atom).replace(' ','')
        if replDict is not None:
            for k,v in replDict.items():
                if s.find(k)>=0:
                    s=s.replace(k,v)
        if noTimeDep:
            s=s.replace('(t)','',)
        return s
    s=''
    indent =''.join([' ']*indent)
    dims=expr.shape
    s+='{}{} = np.zeros(({},{}))\n'.format(indent,varname, dims[0],dims[1])
    for i in np.arange(dims[0]):
        s+='{}{}[{},:] = ['.format(indent,varname,i) + ','.join([cleanPyAtom(expr[i,j]) for j in np.arange(dims[1])]) +']\n' 
    return s

def framematWrap(rottype, order, angles, f, angles_sorted=None):
    if angles_sorted is None:
        angles_sorted=angles
    e_E = ReferenceFrame('E') 
    name=rottype+order
    frame = e_E.orientnew('T', rottype, angles, order) 
    A, omega, omega_g, omega_t, G, Gbar, Ginv, Gbinv = framemat(frame, e_E, angles_sorted)
    
    sAngles = repr(angles_sorted).replace('(t)','').replace('(','').replace(')','')
    f.write('def {}_A({}):\n'.format(name, sAngles))
    f.write('    """ Transformation matrix body to global for rotation of type {} and order {}"""\n'.format(rottype,order))
    f.write(cleanPySmallMat(A, varname='A', noTimeDep=True, indent=4))
    f.write('    return A\n\n')
    
    f.write('def {}_G({}):\n'.format(name, sAngles))
    f.write('    """ Angular velocity matrix such that omega_global = G theta_dot for rotation of type {} and order {}"""\n'.format(rottype,order))
    f.write(cleanPySmallMat(G, varname='G', noTimeDep=True, indent=4))
    f.write('    return G\n\n')
    
    f.write('def {}_Gb({}):\n'.format(name, sAngles))
    f.write('    """ Angular velocity matrix such that omega_local = Gb theta_dot for rotation of type {} and order {}"""\n'.format(rottype,order))
    f.write(cleanPySmallMat(Gbar, varname='Gb', noTimeDep=True, indent=4))
    f.write('    return Gb\n\n')
    
    f.write('def {}_Ginv({}):\n'.format(name, sAngles))
    f.write('    """ Inverse of "G" matrix, such that  theta_dot = Ginv omega_global, for rotation of type {} and order {}"""\n'.format(rottype,order))
    f.write(cleanPySmallMat(Ginv, varname='Ginv', noTimeDep=True, indent=4))
    f.write('    return Ginv\n\n')
    
    f.write('def {}_Gbinv({}):\n'.format(name, sAngles))
    f.write('    """ Inverse of "Gb" matrix, such that  theta_dot = Gbinv omega_local, for rotation of type {} and order {}"""\n'.format(rottype,order))
    f.write(cleanPySmallMat(Gbinv, varname='Gbinv', noTimeDep=True, indent=4))
    f.write('    return Gbinv\n\n')
    
    print('------------- {}'.format(name))
    print(cleantex(angles_sorted))
    print(cleantex(A))
    print(cleantex(G))
    print(cleantex(Gbar))
    print(cleantex(Ginv))
    print(cleantex(Gbinv))
    

with open('_rotations_autogen.py','w') as f:
    f.write('"""\n')
    f.write('Rotation matrices for different conventions: \n')
    f.write(' - BodyZXZ : Euler \n')
    f.write(' - BodyXYZ : Bryant \n')
    f.write(' - BodyZYX :  \n')
    f.write('"""\n')
    f.write('import numpy as np\n')
    f.write('from numpy import cos, sin, tan\n\n')
    
    framematWrap('Body', 'ZXZ', (phi, theta, psi), f)
    framematWrap('Body', 'XYZ', (phi_x, phi_y, phi_z), f)
    framematWrap('Body', 'ZYX', (phi_z, phi_y, phi_x), f, (phi_x, phi_y, phi_z))
    